In [ ]:
import copy
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dateutil.relativedelta import relativedelta

import statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
df = pd.read_csv('data_clean/emergency_situations_clean.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['date'].dtype

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df_day = df.groupby('date')['timeIsoStr'].count().to_frame().rename(columns={'timeIsoStr':'count'})
df_day.head()

In [ ]:
df_day['count'].mean()

In [ ]:
df_day['count'].hist()
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
plt.title('Изменение количества полифакторных ЧС', fontsize=20)
plt.xlabel('Дата', fontsize=15)
plt.ylabel('Количество', rotation=0, labelpad=30, fontsize=10)
df_day['count'].plot();

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(14, 30))
decompose = seasonal_decompose(df_day[['count']], period=365)

ax[0].set_title('Original')
ax[1].set_title('Trend')
ax[2].set_title('Seasonal')
ax[3].set_title('Residual')

decompose.observed.plot(ax=ax[0])
decompose.trend.plot(ax=ax[1])
decompose.seasonal.plot(ax=ax[2])
decompose.resid.plot(ax=ax[3]);

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 15))
df_day['count'].plot(ax=ax[0])
plot_acf(df_day['count'], lags=60, ax=ax[1]);

In [ ]:
corr = acf(df_day['count'], nlags=60, fft=False)

In [ ]:
corr[1], corr[4], corr[8], corr[12]

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

In [ ]:
series = df_day['count'].values

In [ ]:
ar = AutoReg(series, lags=168).fit()

In [ ]:
len(series)

In [ ]:
prediction = ar.predict(start=956, end=956+90-1, dynamic=False)

In [ ]:
df_day.tail()

In [ ]:
import datetime as DT
import pandas as pd

start_date = DT.datetime(2022, 1, 1)
end_date = DT.datetime(2022, 3, 31)

res = pd.date_range(
    min(start_date, end_date),
    max(start_date, end_date)
).strftime('%d/%m/%Y').tolist()

In [ ]:
len(prediction)

In [ ]:
len(res)

In [ ]:
df_pred = pd.DataFrame({'date': res, 'count': prediction}).set_index('date')

In [ ]:
df_pred = pd.concat([df_day, df_pred])

In [ ]:
plt.figure(figsize=(14, 6))
plt.title('Изменение количества полифакторных ЧС', fontsize=20)
plt.xlabel('Дата', fontsize=15)
plt.ylabel('Количество', rotation=0, labelpad=30, fontsize=10)
df_pred['count'].plot();

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(20, 30))
decompose = seasonal_decompose(df_pred[['count']], period=365)

ax[0].set_title('Original')
ax[1].set_title('Trend')
ax[2].set_title('Seasonal')
ax[3].set_title('Residual')

decompose.observed.plot(ax=ax[0])
decompose.trend.plot(ax=ax[1])
decompose.seasonal.plot(ax=ax[2])
decompose.resid.plot(ax=ax[3]);